In [1]:
#cnet_model.ipynb 
# 
#by Joe Hahn
#joe.hahn@oracle.com
#30 July 2018
#
#train a cnet model on the XO dataset

In [2]:
#set system parameters
x_half_width = 0.5
radius = 3.5
box_half_width = 7.1
jitter = 1.0
rn_seed = 15
initial_id = 0

In [3]:
#start time
import time as tm
time_start = tm.time()

In [4]:
#adjust balance of training data so that X class is 2x more abundant that O,B
N_dots = 26700
X_boost = 2.0
import numpy as np
rn_state = np.random.RandomState(seed=rn_seed)
from helper_fns import *
df = make_xo_df(N_dots, initial_id, x_half_width, radius, box_half_width, jitter)
df = rebalance_df(df, X_boost=X_boost)
print df.groupby('class')['id'].count()
train = df
train.tail(5)

class
B    1816
O    1816
X    3633
Name: id, dtype: int64


,id,x,y,r,angle,class,X_score,O_score,B_score
7260,20046,-0.974254,0.457693,1.076408,2.702405,O,0.0,1.0,0.0
7261,6148,0.969538,0.566458,1.122889,0.528763,X,1.0,0.0,0.0
7262,14073,4.867812,-4.248309,6.460938,-0.717546,X,1.0,0.0,0.0
7263,26467,2.961200,4.701584,5.556401,1.008731,X,1.0,0.0,0.0
7264,10876,-0.425876,0.473031,0.636497,2.303784,X,1.0,0.0,0.0


In [5]:
#select x-features and y = target variables = class-scores
x_cols = ['x', 'y']
y_cols = ['O_score', 'X_score', 'B_score']
x_train = train[x_cols].values
y_train = train[y_cols].values
print 'x_train.shape = ', x_train.shape
print 'y_train.shape = ', y_train.shape
x_train

x_train.shape =  (7265, 2)
y_train.shape =  (7265, 3)


array([[ 6.22306129,  5.82059997],
       [-0.59505613,  0.35619103],
       [-2.14182136, -1.41587065],
       ..., 
       [ 4.86781232, -4.24830866],
       [ 2.96120038,  4.70158371],
       [-0.4258758 ,  0.47303123]])

In [6]:
#generate rebalanced validation data
df = make_xo_df(N_dots, initial_id, x_half_width, radius, box_half_width, jitter)
df = rebalance_df(df, X_boost=X_boost)
print df.groupby('class')['id'].count()
x_val = df[x_cols].values
y_val = df[y_cols].values
print 'x_val.shape = ', x_val.shape
print 'y_val.shape = ', y_val.shape
y_val

class
B    1766
O    1766
X    3532
Name: id, dtype: int64
x_val.shape =  (7064, 2)
y_val.shape =  (7064, 3)


array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [7]:
#helper function that builds a cnet classifier
def cnet_classifier(N_input, N_2d, kernel_size, N_output, output_activation_fn):
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, BatchNormalization, Activation, Reshape, Conv2D, MaxPooling2D, Flatten
    model = Sequential()
    #input layer
    N = N_2d**2
    input_shape = (N_input,)
    layer = Dense(N, activation='elu', input_shape=input_shape)
    model.add(layer)
    #reshape input layer
    output_shape = (N_2d, N_2d, 1)
    layer = Reshape(output_shape)
    model.add(layer)
    #add Conv2D + MaxPooling2D layers
    layer = Conv2D(1, kernel_size=kernel_size, padding='same', activation='elu')
    model.add(layer)
    pool_size = kernel_size/2
    layer = MaxPooling2D(pool_size=pool_size, strides=pool_size)
    model.add(layer)
    #add Conv2D + MaxPooling2D layers
    layer = Conv2D(1, kernel_size=kernel_size, padding='same', activation='elu')
    model.add(layer)
    layer = MaxPooling2D(pool_size=pool_size, strides=pool_size)
    model.add(layer)
    #flatten + output layer
    layer = Flatten()
    model.add(layer)
    model.add(Dense(N_output, activation=output_activation_fn))
    #set loss function
    if (output_activation_fn == 'linear'):
        loss_fn = 'mean_squared_error'
    if (output_activation_fn == 'sigmoid'):
        loss_fn = 'categorical_crossentropy'
    model.compile(loss=loss_fn, optimizer='adam')
    return model

In [8]:
#build CNET classification model 
N_input = len(x_cols)
N_2d = 100
kernel_size = 6
N_output = len(y_cols)
output_activation_fn = 'linear'
print 'N_input = ', N_input
print 'N_2d = ', N_2d
print 'kernel_size = ', kernel_size
print 'output_activation_fn = ', output_activation_fn
model = cnet_classifier(N_input, N_2d, kernel_size, N_output, output_activation_fn)
model.summary()

N_input =  2
N_2d =  100
kernel_size =  6
output_activation_fn =  linear


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10000)             30000     
_________________________________________________________________
reshape_1 (Reshape)          (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 1)       37        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 33, 33, 1)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 33, 1)         37        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 1)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 121)               0         
__________

In [ ]:
#train model...is slow on cpu...this runs 20x faster on gpu!
N_training_epochs = 15
batch_size = 2000
fit_history = model.fit(x_train, y_train, epochs=N_training_epochs, batch_size=batch_size, 
    verbose=1, validation_data=(x_val, y_val))

Train on 7265 samples, validate on 7064 samples
Epoch 1/15
7265/7265 [==============================] - 39s 5ms/step - loss: 0.2736 - val_loss: 0.2040
Epoch 2/15
7265/7265 [==============================] - 40s 5ms/step - loss: 0.2003 - val_loss: 0.1861
Epoch 3/15
7265/7265 [==============================] - 40s 5ms/step - loss: 0.1832 - val_loss: 0.1724
Epoch 4/15
7265/7265 [==============================] - 40s 5ms/step - loss: 0.1724 - val_loss: 0.1618
Epoch 5/15
7265/7265 [==============================] - 40s 5ms/step - loss: 0.1620 - val_loss: 0.1539
Epoch 6/15
7265/7265 [==============================] - 45s 6ms/step - loss: 0.1545 - val_loss: 0.1475
Epoch 7/15
7265/7265 [==============================] - 45s 6ms/step - loss: 0.1492 - val_loss: 0.1439
Epoch 8/15
7265/7265 [==============================] - 45s 6ms/step - loss: 0.1463 - val_loss: 0.1419
Epoch 9/15
7265/7265 [==============================] - 42s 6ms/step - loss: 0.1451 - val_loss: 0.1412
Epoch 10/15
7265/7265 [==

In [ ]:
#prep plots
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
sns.set(font_scale=1.5, font='DejaVu Sans')

In [ ]:
#plot accuracy vs training epoch
fig, ax = plt.subplots(1,1, figsize=(15, 6))
xp = fit_history.epoch
yp = fit_history.history['loss']
ax.plot(xp, yp, label='training sample')
yp = fit_history.history['val_loss']
ax.plot(xp, yp, label='validation sample')
ax.set_title('classifier loss versus training epoch')
ax.set_ylabel('loss')
ax.set_xlabel('training epoch')
ax.legend(loc='lower left')
#ax.set_ylim(0.95*np.min(yp), 1.05*np.max(yp[1:]))
plt.savefig('figs/cnet_loss.png')
print 'final validation loss = ', yp[-1]

In [ ]:
#generate class predictions on testing X-O data
N_dots = 20000
df = make_xo_df(N_dots, initial_id, x_half_width, radius, box_half_width, jitter)
x = df[x_cols].values
y = df[y_cols].values
y_pred = model.predict(x)
for idx in range(len(y_cols)):
    y_col = y_cols[idx]
    df[y_col] = y_pred[:, idx]
idx = (df.O_score > df.X_score) & (df.O_score > df.B_score)
df.loc[idx, 'class_pred'] = 'O'
idx = (df.X_score > df.B_score) & (df.X_score > df.O_score)
df.loc[idx, 'class_pred'] = 'X'
idx = (df.B_score > df.O_score) & (df.B_score > df.X_score)
df.loc[idx, 'class_pred'] = 'B'
df.head(5)

In [ ]:
#scatterplot showing model's predicted decision boundaries
xy_rng = (-6.5, 6.5)
sz = 30
f, ax = plt.subplots(figsize=(10,10))
lbl = 'B'
idx = (df['class_pred'] == lbl)
xp = df[idx].x
yp = df[idx].y
p = sns.regplot(xp, yp, ax=ax, fit_reg=False, scatter_kws={'alpha':0.6, 's': sz}, marker='s', label=lbl)
lbl = 'X'
idx = (df['class_pred'] == lbl)
xp = df[idx].x
yp = df[idx].y
p = sns.regplot(xp, yp, ax=ax, fit_reg=False, scatter_kws={'alpha':0.6, 's': sz}, marker='s', label=lbl)
lbl = 'O'
idx = (df['class_pred'] == lbl)
xp = df[idx].x
yp = df[idx].y
p = sns.regplot(xp, yp, ax=ax, fit_reg=False, scatter_kws={'alpha':0.6, 's': sz}, marker='s', label=lbl)
p = ax.set_title('CNET-inferred decision boundary')
p = ax.set_xlabel('x')
p = ax.set_ylabel('y')
p = ax.set_xlim(xy_rng)
p = ax.set_ylim(xy_rng)
plt.savefig('figs/cnet_decision_boundary.png')

In [ ]:
#model accuracy
idx = (df['class'] == df['class_pred'])
accuracy = idx.sum()*1.0/len(idx)
print 'model accuracy = ', accuracy

In [ ]:
#save model
model.save('cnet_model.h5') 

In [ ]:
#done
time_stop = tm.time()
print 'execution time (minutes) = ', (time_stop - time_start)/60.0